In [1]:
import os
import random
import numpy as np
from time import sleep
from IPython.display import clear_output
from collections import deque

import gym
import pylab
import warnings
warnings.filterwarnings('ignore')

In [2]:
import keras
from keras.models import Model, load_model
from keras.layers import Input, Dense
from keras.optimizers import Adam, RMSprop

Using TensorFlow backend.


## Set Environment

In [3]:
ENV_NAME = 'CartPole-v0'
env = gym.make(ENV_NAME).unwrapped; env.seed(90);

In [4]:
print('Environment Display:')
env.reset() # reset environment to a new, random state
env.render()

print('State space {}'.format(env.observation_space))
print('Action space {}'.format(env.action_space))

Environment Display:
State space Box(4,)
Action space Discrete(2)


---